In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shapely.geometry import LineString

In [ ]:
plt.rcParams["font.sans-serif"] = ["SimSun"]
plt.rcParams["axes.unicode_minus"] = False
plt.rcParams["font.size"] = 16

In [ ]:
LIGHT_SOURCE = np.array([0.3101, 0.3162])
SAMPLES = {
    "铁黄": {"loc": np.array([0.4730, 0.4429]), "color": "yellow"},
    "深灰": {"loc": np.array([0.3047, 0.3311]), "color": "darkgray"},
}

In [ ]:
data = pd.read_csv("data.txt", sep="\t", header=0).dropna()

In [ ]:
segments = [
    LineString(
        [
            (data.iloc[i, 1], data.iloc[i, 2]),
            (data.iloc[i + 1, 1], data.iloc[i + 1, 2]),
        ]
    )
    for i in range(len(data) - 1)
]

In [ ]:
def find_intersection(ray_origin, direction_vector, segments):
    ray = LineString([ray_origin, ray_origin + direction_vector * 1000])
    for segment in segments:
        intersection = ray.intersection(segment)
        if not intersection.is_empty:
            return np.array(intersection.xy).T[0]
    return None

In [ ]:
plt.figure(figsize=(16, 10))

# 绘制色品图边界线段
for segment in segments:
    plt.plot(*segment.xy, c="#1f77b4")
    plt.text(
        segment.xy[0][0],
        segment.xy[1][0],
        data[data["x"] == segment.xy[0][0]]["λ"].values[0],
    )

# 绘制光源点
plt.scatter(
    LIGHT_SOURCE[0],
    LIGHT_SOURCE[1],
    s=150,
    c="white",
    marker="o",
    edgecolors="black",
    label="白光",
)

# 绘制样品点及其射线和交点
for label, point in SAMPLES.items():
    # 绘制样品点
    plt.scatter(
        point["loc"][0],
        point["loc"][1],
        s=150,
        c=point["color"],
        marker="s",
        edgecolors="black",
        label=label,
    )

    direction_vector = point["loc"] - LIGHT_SOURCE
    intersection_point = find_intersection(LIGHT_SOURCE, direction_vector, segments)

    # 输出交点坐标并绘制射线和交点
    if intersection_point is not None:
        print(
            f"{label} 交点坐标: {intersection_point[0]:.4f}, {intersection_point[1]:.4f}"
        )
        plt.plot(
            [LIGHT_SOURCE[0], intersection_point[0]],
            [LIGHT_SOURCE[1], intersection_point[1]],
            c=point["color"],
            label=f"{label} 射线",
        )
        plt.scatter(
            intersection_point[0],
            intersection_point[1],
            s=150,
            c=point["color"],
            marker="^",
            edgecolors="black",
            label=f"{label} 交点",
        )

plt.xlabel("$x$")
plt.ylabel("$y$")
plt.legend()

plt.show()